# Ridge regression

In [1]:
import numpy as np
import scipy
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.linear_model import Ridge 

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

data = datasets.load_diabetes()
X_train, y_train = data.data, data.target

In [2]:
def fit_inverse(X, y, alpha=1.0, fit_intercept=False):
    """Direct method using the inverse"""
    # YOUR CODE HERE
    
    if fit_intercept:
        tmp = np.ones((len(X),1))
        X = np.append(tmp, X, axis = 1)
        L = alpha * np.identity(X.shape[1])
        L[0][0] = 0
    else:
        L = alpha * np.identity(X.shape[1])
        
    w = scipy.linalg.inv((X.T).dot(X) + L).dot(np.transpose(X)).dot(y)
    
    return w

In [3]:
w = fit_inverse(X_train, y_train, alpha=0.1)

sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
assert error <= 1e-11
print("prediction error: ", error)

prediction error:  1.8759204781764696e-13


In [4]:
def fit_qr_linear_regression(X, y):
    """ QR approach for linear regression from previous assignement"""
    # YOUR CODE HERE
    (Q, R) = np.linalg.qr(X)
    
    z1 = (np.transpose(Q)).dot(y)
    w = scipy.linalg.solve_triangular(R, z1, lower=False)
    
    return w

def fit_qr(X, y, alpha=1.0, fit_intercept=False):
    """QR approach"""
    # YOUR CODE HERE
    if fit_intercept:
        tmp = np.ones((len(X),1))
        X = np.append(tmp, X, axis = 1)
        v = np.sqrt(alpha) * np.identity(X.shape[1])
        v[0][0] = 0
    else:
        v = np.sqrt(alpha) * np.identity(X.shape[1])
    
    X = np.append(X, v, axis=0)
    
    o = np.zeros(X.shape[1])
    y = np.append(y, o, axis=0)
    
    return fit_qr_linear_regression(X, y)

In [5]:
w = fit_qr(X_train, y_train, alpha=0.1)

sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
#assert error <= 1e-11
print("prediction error: ", error)

prediction error:  1.3307231277965995e-13


In [6]:
def fit_svd(X, y, alpha=1.0, fit_intercept=False):
    """SVD approach"""
    # YOUR CODE HERE
    
    if fit_intercept:
        tmp = np.ones((len(X),1))
        X = np.append(tmp, X, axis = 1)
        
        (U, s, V) = np.linalg.svd(X)
        SS = np.zeros(len(s))

        S = np.zeros_like(np.transpose(X))
        for i in range(len(s)):
            if i==0:
                SS[0] = 1/s[i]
            else:
                SS[i] = s[i] / (s[i]*s[i] + alpha)
        
        S[:X.shape[1], :X.shape[1]] = np.diag(SS)
    else:
        (U, s, V) = np.linalg.svd(X)
    
        S = np.zeros_like(np.transpose(X))
        s = np.array([v / (v*v + alpha) for v in s ])
        S[:X.shape[1], :X.shape[1]] = np.diag(s)
    
    w = np.transpose(V).dot(S).dot(np.transpose(U)).dot(y)
    
    return w

fit_svd(X_train, y_train, 0.1, True)

array([ 152.13348416,    1.30734895, -207.19481363,  489.69108009,
        301.76943732,  -83.46607377,  -70.82809551, -188.68016351,
        115.7127025 ,  443.81405412,   86.74853944])

In [7]:
w = fit_svd(X_train, y_train, alpha=0.1)

sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
#assert error <= 1e-11
print("prediction error: ", error)

prediction error:  2.419419394442948e-13


# Everything in a class

In [8]:
class RidgeRegression():
    def __init__(self, fit_intercept=True, method="inverse", alpha=1.0):
        self.w = 0
        self.fit_intercept = fit_intercept # bias
        self.method = method
        self.alpha = alpha
    
    def fit(self, X, y):
        # YOUR CODE HERE
        if self.method == 'qr':
            self.w = fit_qr(X, y, alpha=self.alpha, fit_intercept=self.fit_intercept)
        elif self.method == 'svd':
            self.w = fit_svd(X, y, alpha=self.alpha, fit_intercept=self.fit_intercept)
        else:
            self.w = fit_inverse(X, y, alpha=self.alpha, fit_intercept=self.fit_intercept)
        
    def predict(self, X):
        # YOUR CODE HERE
        if self.fit_intercept:
            tmp = np.ones((len(X),1))
            X = np.append(tmp, X, axis = 1)
        
        y = X.dot(self.w)
        
        return y

## without bias

In [9]:
# OTHER APPROACHES
sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = RidgeRegression(fit_intercept=False, method="inverse", alpha=0.1)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
assert error <= 1e-11
print("prediction error inverse: ", error)

model_qr = RidgeRegression(fit_intercept=False, method="qr", alpha=0.1)
model_qr.fit(X_train, y_train)
pred_qr = model_qr.predict(X_train)

error_qr = rel_error(pred_qr, sk_pred)
assert error_qr <= 1e-11
print("prediction error qr: ", error_qr)

model_svd = RidgeRegression(fit_intercept=False, method="svd", alpha=0.1)
model_svd.fit(X_train, y_train)
pred_svd = model_svd.predict(X_train)

error_svd = rel_error(pred_svd, sk_pred)
assert error_svd <= 1e-11
print("prediction error svd: ", error_svd)

prediction error inverse:  7.42496493809956e-14
prediction error qr:  1.7943665267072048e-13
prediction error svd:  9.590579711711723e-14


## with bias

In [10]:
# OTHER APPROACHES
sk_model = Ridge(fit_intercept=True, alpha=0.1)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = RidgeRegression(fit_intercept=True, method="inverse", alpha=0.1)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
assert error <= 1e-11
print("prediction error inverse: ", error)

model_qr = RidgeRegression(fit_intercept=True, method="qr", alpha=0.1)
model_qr.fit(X_train, y_train)
pred_qr = model_qr.predict(X_train)

error_qr = rel_error(pred_qr, sk_pred)
#assert error_qr <= 1e-11
print("prediction error qr: ", error_qr)

model_svd = RidgeRegression(fit_intercept=True, method="svd", alpha=0.1)
model_svd.fit(X_train, y_train)
pred_svd = model_svd.predict(X_train)

error_svd = rel_error(pred_svd, sk_pred)
assert error_svd <= 1e-11
print("prediction error svd: ", error_svd)

prediction error inverse:  1.0687672323449546e-15
prediction error qr:  1.0014473512893695e-15
prediction error svd:  4.600019296831778e-15
